In [ ]:
import json
import matplotlib.pyplot as plt

from console import *


def plot_results(experiment, metric, n_iters=3):
    itrs = []
    test_loss = []
    negative_cls_metric = []
    positive_cls_metric = []
    weighted_avg_metric = []
    macro_avg_metric = []

    try:
        for i in range(n_iters):
            itr = experiment[f"iteration-{i}"]
            itrs.append(f"iteration-{i}")

            if metric != "test_loss":
                negative_cls_metric.append(itr["0"][metric])
                positive_cls_metric.append(itr["1"][metric])
                weighted_avg_metric.append(itr["weighted avg"][metric])
                macro_avg_metric.append(itr["macro avg"][metric])
            else:
                test_loss.append(itr["test_loss"])

        if n_iters == 1:
            width = 0.1
        else:
            width = 0.5

        if metric != "test_loss":
            fig, axs = plt.subplots(2, 2)
            fig.suptitle(f"Measurements for {experiment['name']}")
            fig.set_figheight(10)
            fig.set_figwidth(10)

            axs[0][0].set_title(f"Negative class {metric}")
            p = axs[0][0].bar(itrs, negative_cls_metric, width=width, color="tab:blue")
            axs[0][0].bar_label(p, label_type="edge")

            axs[0][1].set_title(f"Positive class {metric}")
            p = axs[0][1].bar(itrs, positive_cls_metric, width=width, color="tab:purple")
            axs[0][1].bar_label(p, label_type="edge")

            axs[1][0].set_title(f"Weighted avg {metric}")
            p = axs[1][0].bar(itrs, weighted_avg_metric, width=width, color="tab:orange")
            axs[1][0].bar_label(p, label_type="edge")

            axs[1][1].set_title(f"Macro avg {metric}")
            p = axs[1][1].bar(itrs, macro_avg_metric, width=width, color="tab:green")
            axs[1][1].bar_label(p, label_type="edge")
        else:
            plt.figure(figsize=(10, 10))
            plt.suptitle(f"Measurements for {experiment['name']}")
            plt.bar(itrs, test_loss, width=width, color="tab:blue")
            plt.show()
    except KeyError as key_err:
        if "iteration" not in str(key_err): 
            print(error(f"Invalid metric key provided --> {key_err}"))
    
   

In [ ]:
report_json = {}
with open("report.json", 'r') as fp:
    report_json = json.load(fp)

In [ ]:
exp_keys = report_json.keys()

dim300 = []
dim200 = []
dim100 = []
dim50  = []

for key in exp_keys:
    if report_json[key]["dim"] == 300:
        dim300.append(key)
    elif report_json[key]["dim"] == 200:
        dim200.append(key)
    elif report_json[key]["dim"] == 100:
        dim100.append(key)
    else:
        dim50.append(key)

F1-Score & 300-dim GloVe

In [ ]:
for exp_key in dim300:
    exp = report_json[exp_key]
    n_iters = 0
    for k in exp.keys():
        if "iteration" in k:
            n_iters += 1
    plot_results(experiment=exp, metric="f1-score", n_iters=n_iters)

In [ ]:
for exp_key in dim300:
    exp = report_json[exp_key]
    n_iters = 0
    for k in exp.keys():
        if "iteration" in k:
            n_iters += 1
    plot_results(experiment=exp, metric="recall", n_iters=n_iters)

In [ ]:
for exp_key in dim300:
    exp = report_json[exp_key]
    n_iters = 0
    for k in exp.keys():
        if "iteration" in k:
            n_iters += 1
    plot_results(experiment=exp, metric="precision", n_iters=n_iters)

In [ ]:
for exp_key in dim300:
    exp = report_json[exp_key]
    n_iters = 0
    for k in exp.keys():
        if "iteration" in k:
            n_iters += 1
    plot_results(experiment=exp, metric="test_loss", n_iters=n_iters)